# Sources

Modified from an original project by nik-davis at [github](https://github.com/nik-davis/steam-data-science-project)


# SteamSpy Data Cleaning

*This forms part of a larger series of posts for my [blog](http://nik-davis.github.io) on downloading, processing and analysing data from the steam store. [See all posts here](http://nik-davis.github.io/tag/steam).*

In [1]:
# Original at
# https://github.com/nik-davis/steam-data-science-project/blob/master/notebooks/4-data-cleaning-steamspy.ipynb
# 

<!-- PELICAN_BEGIN_SUMMARY -->

Welcome to the final part of the data cleaning process. Once we're finished here we'll be ready to move on to exploring and analysing the data.

As a quick re-cap, so far we have downloaded information on games from the Steam Store via the Steam API and SteamSpy API. We have cleaned and processed the data from the Steam API, and in this section we'll walkthrough cleaning data downloaded from the SteamSpy API. The overall goal of this project is to collect, clean and analyse data from the Steam Store with the idea of advising a fictional game developer or company.

The previous posts went into great depth about the decisions made and methods used. This post will still go over a number of decisions, but will be more in the style of a brief overview than full discussion.

<!-- PELICAN_END_SUMMARY -->

## Import Libraries and Inspect Data

Let's begin by importing the necessary libraries and inspecting the raw downloaded data.

In [2]:
# standard library imports
from ast import literal_eval
import itertools
# import time
# import re

# third-party imports
import numpy as np
import pandas as pd

# customisations
pd.set_option("max_columns", 100)

In [3]:
raw_steamspy_data = pd.read_csv('../data/download/steamspy_data.csv')
raw_steamspy_data.head(1)

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags
0,10,Counter-Strike,Valve,Valve,NaN,183324,4765,0,"10,000,000 .. 20,000,000",10560,633,185,1007,999.0,999.0,0.0,"English, French, German, Italian, Spanish - Sp...",Action,13403,"{'Action': 5366, 'FPS': 4789, 'Multiplayer': 3..."


There are a lot of columns here that will overlap with columns we already dealt with in the Steam data. This is fine because it means we don't have to clean them here, and can simply use the columns from the other dataset.

A few columns in this data that could be valuable are `positive`, `negative`, `userscore`, `owners` and `average_forever`. We'll begin by looking at the null counts for each column.

In [4]:
raw_steamspy_data.isnull().sum()

appid                  0
name                  11
developer            216
publisher            210
score_rank         47353
positive               0
negative               0
userscore              0
owners                 0
average_forever        0
average_2weeks         0
median_forever         0
median_2weeks          0
price                 29
initialprice          22
discount              22
languages             87
genre                261
ccu                    0
tags                   0
dtype: int64

Most of the columns look fine, with only a few tens or hundreds of missing values, or none at all. The only column we'll remove for certain is the `score_rank` column, as it has far too many null values to be useful.

## Handling missing values

There are some columns we'll be removing, as they're present in the other dataset. Before we do we can investigate the missing values, to decide whether we should remove these rows first. We don't need to go through every column, but just look at a few individual ones.

In the `name` column, we have five rows with missing data, and four more rows where the name is 'none'. We'll certainly remove all 9 of these rows.

In [5]:
raw_steamspy_data[raw_steamspy_data['name'].isnull()]

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags
478,17760,NaN,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,0.0,0.0,0.0,NaN,NaN,0,[]
7171,396420,NaN,NaN,Aterdux Entertainment,NaN,25,13,0,"0 .. 20,000",0,0,0,0,0.0,0.0,0.0,"English, Not supported",NaN,0,{'Free to Play': 6}
9406,460250,NaN,Jeroen Wimmers,Jeroen Wimmers,NaN,69,5,0,"0 .. 20,000",0,0,0,0,899.0,899.0,0.0,"English, French, Italian, German, Spanish - Sp...","Casual, Indie",0,"{'Experimental': 89, 'Minimalist': 83, 'Relaxi..."
10855,506630,NaN,NaN,Big Fish Games,NaN,1,0,0,"0 .. 20,000",0,0,0,0,0.0,0.0,0.0,"English, Not supported",NaN,0,"{'Action': 21, 'Adventure': 21}"
13490,576960,NaN,Nuclear Tales,NaN,NaN,110,26,0,"20,000 .. 50,000",0,0,0,0,299.0,1499.0,80.0,"English, Spanish - Spain, French, German","Adventure, Casual, Indie, RPG, Simulation, Str...",3,"{'Adventure': 376, 'Choose Your Own Adventure'..."
21886,806160,NaN,Paleno Games,Paleno Games,NaN,5,19,0,"0 .. 20,000",0,0,0,0,99.0,99.0,0.0,"English, French, Italian, German, Spanish - Sp...","Action, Adventure, Casual, Indie",0,"{'Action': 22, 'Indie': 22, 'Casual': 21, 'Adv..."
21888,806220,NaN,Intermediaware,Intermediaware,NaN,16,13,0,"0 .. 20,000",0,0,0,0,399.0,399.0,0.0,English,"Action, Indie",0,"{'Action': 33, 'Indie': 32, ""Shoot 'Em Up"": 14..."
27453,965340,NaN,2nd Studio,2nd Studio,NaN,55,7,0,"0 .. 20,000",0,0,0,0,199.0,199.0,0.0,"English, German, Danish, Japanese, Russian, Si...","Action, Indie, Simulation",0,"{'Indie': 34, 'Action': 31, 'Sexual Content': ..."
31049,1071920,NaN,Ultimo Games,Ultimo Games,NaN,19,0,0,"0 .. 20,000",0,0,0,0,399.0,399.0,0.0,English,"Action, Indie",0,"{'Indie': 32, 'Action': 32, 'Retro': 14, ""Shoo..."
35685,1216770,NaN,NaN,Unusual Games,NaN,1,0,0,"0 .. 20,000",0,0,0,0,0.0,0.0,0.0,"English, French",NaN,0,"{'Casual': 104, 'Indie': 98, 'RPGMaker': 94, '..."


In [6]:
raw_steamspy_data[raw_steamspy_data['name'] == 'none']

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags
4922,339860,none,NaN,Green Sauce Games,NaN,84,36,0,"50,000 .. 100,000",288,0,307,0,0.0,0.0,0.0,English,NaN,0,"{'Indie': 53, 'RPG': 33, 'Adventure': 32, 'Mas..."
6739,385020,none,none,Valsar,NaN,13,33,0,"20,000 .. 50,000",278,0,278,0,0.0,0.0,0.0,"English, French, Italian, German, Spanish - Sp...","Casual, Indie, Massively Multiplayer, Strategy",1,"{'Indie': 47, 'Strategy': 31, 'Casual': 21, 'M..."
7279,398970,none,none,none,NaN,8,23,0,"20,000 .. 50,000",267,0,267,0,0.0,0.0,0.0,English,"Adventure, Indie, Massively Multiplayer, RPG, ...",0,"{'Massively Multiplayer': 38, 'Strategy': 23, ..."


There are 197 rows with missing developer data, and 280 rows with missing publisher data. It's conceivable that a small studio may self-publish, but we'll say that a title must have a developer for it to be kept. We can remove the rows with missing developer data, but then we'll be removing both of these columns as this information is already provided in the Steam data.

In [7]:
raw_steamspy_data[raw_steamspy_data['developer'].isnull()].sample(5, random_state=0)

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags
32126,1105150,Reverse Me! Rez/Ru,NaN,10F,NaN,3,0,0,"0 .. 20,000",0,0,0,0,0.0,0.0,0.0,"English, French",NaN,0,"{'Casual': 103, 'Indie': 97, 'Visual Novel': 9..."
1588,209340,Ride to Hell: Retribution,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,0.0,0.0,0.0,NaN,NaN,1,[]
2441,253650,Sparkle 2 Evo,NaN,Forever Entertainment S. A.,NaN,1414,547,0,"200,000 .. 500,000",178,0,183,0,499.0,499.0,0.0,English,NaN,1,"{'Casual': 79, 'Indie': 68, 'Simulation': 57, ..."
20998,783140,Removed,NaN,Slitherine Ltd.,NaN,25,16,0,"200,000 .. 500,000",119,0,142,0,0.0,0.0,0.0,"English, Not supported",NaN,0,"{'Simulation': 1105, 'Action': 35, 'Indie': 33}"
16373,663920,Died Of Fear,NaN,Type B Negative,NaN,17,15,0,"0 .. 20,000",0,0,0,0,99.0,99.0,0.0,English,NaN,0,"{'Casual': 32, 'Indie': 31, '2D': 14, 'Side Sc..."


In [8]:
raw_steamspy_data[raw_steamspy_data['publisher'].isnull()].sample(5, random_state=0)

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags
635,29017,Blood Bowl 2 - Review,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,NaN,0.0,0.0,NaN,NaN,0,[]
7354,401510,Lavapools - Arcade Frenzy,"Tavrox, Visumeca",NaN,NaN,15,1,0,"0 .. 20,000",0,0,0,0,499.0,499.0,0.0,"English, French","Action, Indie",0,"{'2D Platformer': 177, 'Bullet Hell': 171, 'Co..."
1406,200350,Starvoid,NaN,NaN,NaN,0,0,0,"0 .. 20,000",1,0,1,0,0.0,0.0,0.0,NaN,NaN,1,[]
392,13260,Unreal Development Kit,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,0.0,0.0,0.0,NaN,NaN,88,[]
26893,948340,Journey Through Memories,"Niken Hertanto, Inkan Hertanto, Juan Rosales, ...",NaN,NaN,4,1,0,"0 .. 20,000",0,0,0,0,99.0,99.0,0.0,English,Indie,0,{'Indie': 22}


There are a small number of rows with missing price data. This is another column we don't need, (as well as `initialprice` and `discount`), as we already have that data available in the other dataset. 

It looks like these rows have a lot of other missing information as well, so if they haven't already been removed when cleaning other columns we'll remove them here.

In [9]:
raw_steamspy_data[raw_steamspy_data['price'].isnull()].sample(5, random_state=0)

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags
4279,321210,Blade Symphony Beta,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,[]
17885,700580,Rust - Staging Branch,NaN,NaN,NaN,0,0,0,"0 .. 20,000",706,0,2,0,NaN,NaN,NaN,NaN,NaN,5,[]
22148,813350,Ben 10 VR,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,[]
15334,630790,PSYCHO-PASS,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,NaN,NaN,NaN,NaN,NaN,1,[]
11794,530940,BIOHAZARD 7 resident evil グロテスクVer.,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,NaN,NaN,NaN,NaN,NaN,20,[]


Again there are a small number of rows with missing language data. There are a couple of bigger titles here, but most are missing other data also. We'll go ahead and remove these rows and this should leave us with most troublesome rows dealt with.

In [10]:
raw_steamspy_data[raw_steamspy_data['languages'].isnull()].sample(5, random_state=0)

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags
255,8740,Puzzlegeddon,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,0.0,0.0,0.0,NaN,NaN,1,[]
803,35470,The Ball Beta,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,0.0,0.0,0.0,NaN,NaN,0,[]
3453,293360,Second Chance Heroes,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,0.0,0.0,0.0,NaN,NaN,0,[]
1807,222320,Football Manager 2013 Asia,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,0.0,0.0,0.0,NaN,NaN,3,[]
13840,587030,Bunker Busters Steamworks Test,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,[]


## Remove unwanted columns

As mentioned, there are a number of columns present in the Steam dataset, so we can remove any of those with crossover. It makes sense that these exist because SteamSpy's data is retrieved by looking at Steam's data. We'll remove the `score_rank` column as it contains too many missing values, and we'll remove the `userscore` column as most rows have a value of 0 here, so it won't be useful.

In [11]:
raw_steamspy_data['userscore'].value_counts().head()

0      47353
100        4
95         3
46         2
68         2
Name: userscore, dtype: int64

There are also three columns - `average_2weeks`, `median_2weeks`, (average and median playtime in the last 2 weeks) and `ccu` (Peak conccurrent users yesterday) - that provide information only relevant to the time the data was collected. Going forward this isn't going to be useful, as we're not interested in how games were performing at that specific time, so we'll remove those.

In [12]:
drop_cols = [
    'score_rank', # too many missing values
    'userscore', # too little variance (most have 0)
    'genre', 'developer', 'publisher', 'price', 'initialprice', 'discount', 'languages'# provided by Steam data
    # 'average_2weeks', 'median_2weeks', 'ccu' # not interested in temporally specific columns
]

One more column we could remove is the `languages` column, as we already have the `english` column in the Steam Data. Here in the SteamSpy data, languages is made up of a comma-separated list of all supported languages. This provides us with some more information so we can keep it for now.

In [13]:
raw_steamspy_data['languages'].head()

0    English, French, German, Italian, Spanish - Sp...
1    English, French, German, Italian, Spanish - Sp...
2    English, French, German, Italian, Spanish - Spain
3    English, French, German, Italian, Spanish - Sp...
4                      English, French, German, Korean
Name: languages, dtype: object

## Process & Export Tags Column

There are a couple of columns that require more specific handling, and the `tags` column is one of them. There will be a lot of overlap with the `genres` column, but the tags might provide just enough unique information to be worthwhile keeping.

Below we separate out the tags data for experimentation, then have a look a the first few rows. Each tag for a game has an associated number, corresponding to the amount of votes that game has for that tag. A game's tags with higher votes means that more people think it is a relevant tag for that title.

In [14]:
tags_df = raw_steamspy_data[['appid', 'tags']]

tags_df.head(2)
tags = tags_df['tags']

Just by looking at the first row you can see that there are probably going to be far too many tags to include without disrupting the dataset. We have a couple of options here. We could include just the names of the top 3 or 5 tags for a game - which should be easy enough as it looks like the tags are already in order from most votes to least - or we could create new columns for each tag, and the number of votes as the row values, like this:

appid | name | action | fps | multiplayer
--- | --- | --- | --- | ---
10 | Shoot 'em up | 2681 | 2048 | 0
20 | Big Brawl | 208  | 0 | 172


In a way, we'll do both.

Because including the full tag information in the dataset may make it bulky and unwieldy, we'll only include the top three tags in the core dataset. As we did when cleaning the Steam Data, we'll also export the full tag data as a separate file. This means that if we want to use it as part of our analysis we can import and merge the data quite easily.

Before we can do that, we have to figure out how to handle and process the data. As we've seen previously, the data in each row is stored as a string containing a dictionary, so we have to use `literal_eval` to get python to "recognise" the dictionary first.

In [15]:
eval_row = literal_eval(tags[0])

print(eval_row['Action'])
print(eval_row['FPS'])

5366
4789


We also have to figure out how to get a list of all the different unique tags from across the dataset. As each row only contains the tags that it has votes for, we don't know from the outset the tag names or even how many different tags there are.

We could loop through the data and build a list of all the unique tag names using a dictionary or something similar, but instead we'll achieve the same thing much more easily using the [chain](https://docs.python.org/3/library/itertools.html?highlight=itertools#itertools.chain) function from itertools. As seen below, it will take a variety of iterables and unpack them into a single sequence (we have to use list() here to force the generator to evaluate). We can then use the set function to obtain just the unique values from the returned sequence.

As you can see below, we can pass lists, tuples and dictionaries and they will all be handled. Also notice how only the key from the dictionary is returned (d) and not the value inside the key (e). This is really useful and means we can just pass the evaluated tags data as a series, and itertools will just extract the individual keys.

In [16]:
values = [
    ['a', 'b'], # list
    ('b', 'c'), # tuple
    {'d': 'e'}  # dictionary
]

list(itertools.chain(*values))

['a', 'b', 'b', 'c', 'd']

In [17]:
set(itertools.chain(*values))

{'a', 'b', 'c', 'd'}

If we tried to define a function now, we might run into a problem. It turns out not all of the rows have data stored as dictionaries. Seen below, there are a few hundred rows with an empty list.

In [18]:
tags[tags == '[]'].shape[0]

360

This doesn't cause an issue when we generate our set of unique tag names, as any empty lists are ignored.

In [19]:
parsed_tags = tags.apply(lambda x: literal_eval(x))

cols = set(itertools.chain(*parsed_tags))

print('Number of unique tags:', len(cols))
print('\nFirst few tags:', sorted(list(cols))[:5])

Number of unique tags: 428

First few tags: ['1980s', "1990's", '2.5D', '2D', '2D Fighter']


However to create the new dataframe from the tags and votes, we need to look inside the dictionaries in each row. We'll expand the logic to parse the tags using the `isintance` function to check each row for its type. If it is a dictionary, we return the dictionary. If it is a list, we'll return an empty dictionary `{}`. We'll also include an extra check in case there is anything else in the rows we haven't found yet. 

Next, we loop through the tags stored in `cols`, standardise the format of the new column names, then extract the vote count of each tag by checking if the tag name is in each row's dictionary of tags.

In [20]:
def parse_tags(x):
    x = literal_eval(x)
    
    if isinstance(x, dict):
        return x
    elif isinstance(x, list):
        return {}
    else:
        raise TypeError('Something other than dict or list found')

parsed_tags = tags.apply(parse_tags)
        
tag_data = pd.DataFrame()

for col in sorted(cols):
    # standardise column names
    col_name = col.lower().replace(' ', '_').replace('-', '_').replace("'", "")

    # check if column in row's dictionary of tags and return that value if it is, or 0 if it isn't
    tag_data[col_name] = parsed_tags.apply(lambda x: x[col] if col in x.keys() else 0)

tag_data.head()

,1980s,1990s,2.5d,2d,2d_fighter,2d_platformer,360_video,3d,3d_fighter,3d_platformer,3d_vision,4_player_local,4x,6dof,8_bit_music,atv,abstract,action,action_rpg,action_rts,action_roguelike,action_adventure,addictive,adventure,agriculture,aliens,alternate_history,ambient,america,animation_&_modeling,anime,arcade,archery,arena_shooter,artificial_intelligence,assassin,asymmetric_vr,asynchronous_multiplayer,atmospheric,audio_production,auto_battler,automation,automobile_sim,bmx,base_building,baseball,based_on_a_novel,basketball,battle_royale,beat_em_up,...,time_manipulation,time_travel,top_down,top_down_shooter,touch_friendly,tower_defense,trackir,trading,trading_card_game,traditional_roguelike,trains,transhumanism,transportation,trivia,turn_based,turn_based_combat,turn_based_strategy,turn_based_tactics,tutorial,twin_stick_shooter,typing,underground,underwater,unforgiving,utilities,vr,vr_only,vampire,vehicular_combat,video_production,vikings,villain_protagonist,violent,visual_novel,voice_control,voxel,walking_simulator,war,wargame,warhammer_40k,web_publishing,well_written,werewolves,western,word_game,world_war_i,world_war_ii,wrestling,zombies,e_sports
0,256,1177,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5366,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,222,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,63,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1168
1,0,131,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,745,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,45,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,157,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,149,0,0,0,0,0,0,0,12,246,0,0,0
3,0,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,628,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,44,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,132,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,320,0,0,0,0,0,112,0,170,0,0,0,0,0,0,0,0,0,0,0,0,104,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [21]:
def export_data(df, filename):
    """Export dataframe to csv file, filename prepended with 'steam_'.
    
    filename : str without file extension
    """
    filepath = '../data/exports/steamspy_' + filename + '.csv'
    
    df.to_csv(filepath, index=False)
    
    print_name = filename.replace('_', ' ')
    print("Exported {} to '{}'".format(print_name, filepath))

In [22]:
def process_tags(df):
    """Split platforms column into separate boolean columns for each platform."""
    # evaluate values in platforms column, so can index into dictionaries
    df = df.copy()
    df['tags'] = df['tags'].apply(parse_tags)
    df = df.join(pd.DataFrame(df.pop('tags').values.tolist()))
    
    return df

tags_df = process_tags(raw_steamspy_data[['appid', 'tags']])

tags_df.fillna(0)
export_data(tags_df, 'tags')

Exported tags to '../data/exports/steamspy_tags.csv'


We'll have to add in the AppIDs, but apart from that this makes up the tag data that we'll export. Next we have to figure out how to extract just the top three tags for use in the core dataset.

It turns out this is a relatively simple process. As the keys are in order of highest votes to least, we can just extract the first 3 keys for each row. This may not be the safest approach, as dictionaries don't necessarily preserve key order, but it's fine for our purposes. We can then join these keys on a semicolon to create a list of the tags.

In [23]:
def parse_tags(x):
    x = literal_eval(x)

    if isinstance(x, dict):
        return ';'.join(list(x.keys())[:3])
    else:
        return np.nan
    
tags.apply(parse_tags).head()

0          Action;FPS;Multiplayer
1          Action;FPS;Multiplayer
2    FPS;World War II;Multiplayer
3              Action;FPS;Classic
4              FPS;Action;Classic
Name: tags, dtype: object

## Handle Owners Column

The final column we'll look at before defining a function to perform the cleaning is the `owners` column. This column is made up of the lower and upper bound of an estimation for the amount of owners for each title. For privacy reasons, SteamSpy can't get exact figures from Steam (though it [used to](https://www.polygon.com/2018/4/12/17229752/steam-spy-charts-new-privacy-rules-valve)), so we'll have to make do with the figures we have here.

In [24]:
owners = raw_steamspy_data['owners']
owners.head()

0    10,000,000 .. 20,000,000
1      2,000,000 .. 5,000,000
2     5,000,000 .. 10,000,000
3     5,000,000 .. 10,000,000
4     5,000,000 .. 10,000,000
Name: owners, dtype: object

We have a couple of options for how to deal with the data. We could remove the commas and split the data, keeping the lower or upper bound.

In [25]:
owners_split = owners.str.replace(',', '').str.split(' .. ')
owners_split.apply(lambda x: int(x[0])).head()

0    10000000
1     2000000
2     5000000
3     5000000
4     5000000
Name: owners, dtype: int64

We could calculate the mid-point between the points, and keep that.

In [26]:
owners_split.apply(lambda x: (int(x[0]) + int(x[1])) // 2).head()

0    15000000
1     3500000
2     7500000
3     7500000
4     7500000
Name: owners, dtype: int64

Or we could reformat the data slightly, but pretty much keep it as is.

In [27]:
owners.str.replace(',', '').str.replace(' .. ', '-').head()

0    10000000-20000000
1      2000000-5000000
2     5000000-10000000
3     5000000-10000000
4     5000000-10000000
Name: owners, dtype: object

We'll go with this last option. It allows us to keep the ranges and also easily modify the data in the future.

## Define Function

We're now ready to define the functions to clean the data. We'll wrap all the logic inside a `process` function, and for most of the columns there isn't much code to write so we can confine it here. The only logic we'll separate out is for processing the tags column, as it is a more complex endeavour.

In the next code cell we clean the data, export the full tags data to file, and inspect the clean dataframe. After that we inspect the exported data, verifying everything went as planned.

In [28]:
def process_tags(df, export=False):
    if export: 
        
        tag_data = df[['appid', 'tags']].copy()
        
        def parse_export_tags(x):
            x = literal_eval(x)

            if isinstance(x, dict):
                return x
            elif isinstance(x, list):
                return {}
            else:
                raise TypeError('Something other than dict or list found')

        tag_data['tags'] = tag_data['tags'].apply(parse_export_tags)

        cols = set(itertools.chain(*tag_data['tags']))

        for col in sorted(cols):
            col_name = col.lower().replace(' ', '_').replace('-', '_').replace("'", "")

            tag_data[col_name] = tag_data['tags'].apply(lambda x: x[col] if col in x.keys() else 0)

        tag_data = tag_data.drop('tags', axis=1)

        tag_data.to_csv('../data/exports/steamspy_tag_data.csv', index=False)
        print("Exported tag data to '../data/exports/steamspy_tag_data.csv'")
        
        
    def parse_tags(x):
        x = literal_eval(x)
        
        if isinstance(x, dict):
            return ';'.join(list(x.keys())[:3])
        else:
            return np.nan
    
    df['tags'] = df['tags'].apply(parse_tags)
    
    # rows with null tags seem to be superseded by newer release, so remove (e.g. dead island)
    df = df[df['tags'].notnull()]
    
    return df


In [29]:

def process(df):
    df = df.copy()
    
    # handle missing values
    df = df[(df['name'].notnull()) & (df['name'] != 'none')]
    # df = df[df['developer'].notnull()]
    # df = df[df['languages'].notnull()]
    # df = df[df['price'].notnull()]
    drop_cols = [
        'tags',
        'name',
        'score_rank', # too many missing values
        'userscore', # too little variance (most have 0)
        'genre', 'developer', 'publisher', 'price', 'initialprice', 'discount', 'languages'# provided by Steam data
        # 'average_2weeks', 'median_2weeks', 'ccu' # not interested in temporally specific columns
    ]
    # remove unwanted columns
    df = df.drop(drop_cols, axis=1)
    
    # keep top tags, exporting full tag data to file
    # df = process_tags(df, export=True)
    
    # reformat owners column
    # df['owners'] = df['owners'].str.replace(',', '').str.replace(' .. ', '-')
    
    return df


steamspy_data = process(raw_steamspy_data)
steamspy_data
export_data(steamspy_data, 'data_clean')

Exported data clean to '../data/exports/steamspy_data_clean.csv'


In [30]:
# inspect tag data
pd.read_csv('../data/exports/steamspy_tags.csv').head()

,appid,1980s,1990's,2.5D,2D,2D Fighter,2D Platformer,360 Video,3D,3D Fighter,3D Platformer,3D Vision,4 Player Local,4X,6DOF,8-bit Music,ATV,Abstract,Action,Action RPG,Action RTS,Action Roguelike,Action-Adventure,Addictive,Adventure,Agriculture,Aliens,Alternate History,Ambient,America,Animation & Modeling,Anime,Arcade,Archery,Arena Shooter,Artificial Intelligence,Assassin,Asymmetric VR,Asynchronous Multiplayer,Atmospheric,Audio Production,Auto Battler,Automation,Automobile Sim,BMX,Base-Building,Baseball,Based On A Novel,Basketball,Battle Royale,...,Time Manipulation,Time Travel,Top-Down,Top-Down Shooter,Touch-Friendly,Tower Defense,TrackIR,Trading,Trading Card Game,Traditional Roguelike,Trains,Transhumanism,Transportation,Trivia,Turn-Based,Turn-Based Combat,Turn-Based Strategy,Turn-Based Tactics,Tutorial,Twin Stick Shooter,Typing,Underground,Underwater,Unforgiving,Utilities,VR,VR Only,Vampire,Vehicular Combat,Video Production,Vikings,Villain Protagonist,Violent,Visual Novel,Voice Control,Voxel,Walking Simulator,War,Wargame,Warhammer 40K,Web Publishing,Well-Written,Werewolves,Western,Word Game,World War I,World War II,Wrestling,Zombies,e-sports
0,10,256.0,1177.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5366.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,222.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1168.0
1,20,NaN,131.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,745.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,157.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,149.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,246.0,NaN,NaN,NaN
3,40,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,628.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,50,NaN,132.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,320.0,NaN,NaN,NaN,NaN,NaN,112.0,NaN,170.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,104.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
